# Overlaying image segmentation mask without associated image

## Environment hardware

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
# Some basic setup:
# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
from google.colab import files
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from PIL import Image

import gdown
import zipfile
import shutil
import time
import functools

from zipfile import ZipFile


## Define directories and link to gdrive

In [ ]:
os.mkdir('Masks')
os.mkdir('Images')
os.mkdir('Outputs')
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Define images directory

Usually, there is only one images directory (dataset inputs) for all the segmentations masks outputs directory (inference output masks or ground-truth masks.

In [ ]:
# Extract images

# path of zip archive containing images jpeg
images_archive = '/content/drive/MyDrive/MVA/RecVis/Project/Visualization/DAVIS/davis_validation_jpeg.zip'


images_filename, images_file_extension = os.path.splitext(images_archive)
images_basename = os.path.basename(os.path.normpath(images_filename))

path_to_images = os.path.join('/content/Images/', images_basename)

if True:
  with ZipFile(images_archive, 'r') as zipObj:
    # Extract all the contents of zip file in different directory
    zipObj.extractall('Images')
    print('Images archive is unzipped in Images folder') 
elif os.path.isdir(images_archive):
  shutil.copytree(
      path_to_images,
      path_to_images
  )


Images archive is unzipped in Images folder


## Define masks directory

In [ ]:
# Extract ground-truth masks

# path of zip archive containing ground truth png masks
masks_gt_archive = '/content/drive/MyDrive/MVA/RecVis/Project/Visualization/DAVIS/davis_val_annot_gt.zip'
masks_gt_filename, masks_file_extension = os.path.splitext(masks_gt_archive)
masks_gt_basename = os.path.basename(os.path.normpath(masks_gt_filename))

path_to_masks_gt = os.path.join('/content/Masks/', masks_gt_basename)

if zipfile.is_zipfile(masks_gt_archive):
  with ZipFile(masks_gt_archive, 'r') as zipObj:
    # Extract all the contents of zip file in different directory
    zipObj.extractall('Masks')
    print('Maks gt archive is unzipped in Mask folder') 
elif os.path.isdir(masks_gt_archive):
  shutil.copytree(
      masks_gt_archive,
      path_to_masks_gt
  )


Maks gt archive is unzipped in Mask folder


In [ ]:
# Extract masks

# path of zip archive containing inferred png masks
masks_archive = '/content/drive/MyDrive/MVA/RecVis/Project/Visualization/DAVIS/output_davis.zip'
masks_filename, masks_file_extension = os.path.splitext(masks_archive)
masks_basename = os.path.basename(os.path.normpath(masks_filename))

path_to_masks = os.path.join('/content/Masks/', masks_basename)

if zipfile.is_zipfile(masks_archive):
  with ZipFile(masks_archive, 'r') as zipObj:
    # Extract all the contents of zip file in different directory
    zipObj.extractall('Masks')
    print('Maks archive is unzipped in Mask folder') 
elif os.path.isdir(masks_archive):
  shutil.copytree(
      masks_archive,
      path_to_masks
  )


Maks archive is unzipped in Mask folder


In [ ]:
path_to_masks

'/content/Masks/output_davis'

## Load video sequence and images

In [ ]:
## OPTIONS
video_sequences = ['bike-packing', 'blackswan', 'bmx-trees', 'breakdance', 'camel', 'car-roundabout',
'car-shadow', 'cows', 'dance-twirl', 'dog', 'dogs-jump', 'drift-chicane',
'drift-straight', 'goat', 'gold-fish', 'horsejump-high', 'india', 'judo',
'kite-surf', 'lab-coat', 'libby', 'loading', 'mbike-trick', 'motocross-jump',
'paragliding-launch', 'parkour', 'pigs', 'scooter-black', 'shooting',
 'soapbox']
alpha = 0.5  # transparency parameter
stride = 3  # Browse each {stride} images
save_im_blend = True  # Save segmentation
save_im_blend_gt = False  # Save ground-truth segmentation
show_gt_mask = False  # Show ground truth mask

In [ ]:
for  video_sequence in video_sequences:
    video_ims_path = os.path.join(path_to_images, video_sequence)
    video_masks_path = os.path.join(path_to_masks, video_sequence)
    video_masks_gt_path = os.path.join(path_to_masks_gt, video_sequence)

    list_ims_files = os.listdir(video_ims_path)
    list_masks_files = os.listdir(video_masks_path)
    list_ims_files.sort()
    list_masks_files.sort()


    for i, (im_filename, mask_filename) in enumerate(list(zip(list_ims_files, list_masks_files))[::stride]):
        print('=====================================================================')
        print('=====================================================================')  
        print(f'Video {video_sequence} frame: {stride*i}')
        im = Image.open(os.path.join(video_ims_path, im_filename))
        mask = Image.open(os.path.join(video_masks_path, mask_filename))
        if show_gt_mask:
          mask_gt = Image.open(os.path.join(video_masks_gt_path, mask_filename))

        # display(im)
        # display(mask)

        # im_blend = Image.blend(im.convert('RGB'), mask.convert('RGB'), 0.5)
        # maskx = mask.copy().convert("L")
        # print(np.array(maskx).shape)

        # im_blend = Image.composite(mask, im, maskx)
        # display(im_blend)



        im = np.array(im)
        mask = np.array(mask.convert('RGB'))
        im_blend = np.array(im)

        mask_alpha = np.any(mask > 0., axis=2)
        im_blend[mask_alpha] = (
            alpha*im[mask_alpha]
            + (1-alpha)*mask[mask_alpha]
        )

        if show_gt_mask:
            mask_gt = np.array(mask_gt.convert('RGB'))
            im_blend_gt = np.array(im)
            mask_gt_alpha = np.any(mask_gt > 0., axis=2)
            im_blend_gt[mask_gt_alpha] = (
                alpha*im[mask_gt_alpha]
                + (1-alpha)*mask_gt[mask_gt_alpha]
            )


        # Visualization
        cv2_imshow(im[:, :, ::-1])
        print('---------------------------------------------------------------------')
        print('Mask infered')
        cv2_imshow(mask[:, :, ::-1])
        print('---------------------------------------------------------------------')
        print('Mask ground-truth')
        if show_gt_mask:
            cv2_imshow(mask_gt[:, :, ::-1])
        print('---------------------------------------------------------------------')
        print('Image final infered')
        cv2_imshow(im_blend[:, :, ::-1])
        print('---------------------------------------------------------------------')
        print('Image final ground-truth')
        if show_gt_mask:
            cv2_imshow(im_blend_gt[:, :, ::-1])
        if save_im_blend:
            os.makedirs(os.path.join('Outputs/', masks_basename, video_sequence),
                        exist_ok=True)
            cv2.imwrite(
                os.path.join('Outputs/', masks_basename, video_sequence, im_filename),
                im_blend[:, :, ::-1]
            )

        if show_gt_mask and save_im_blend_gt:
            os.makedirs(os.path.join('Outputs/', masks_gt_basename, video_sequence),
                        exist_ok=True)
            cv2.imwrite(
                os.path.join('Outputs/', masks_gt_basename, video_sequence, im_filename),
                im_blend_gt[:, :, ::-1]
            )
        
        time.sleep(4.)


## Download results

In [ ]:
!zip -r /content/Outputs_visualization.zip Outputs
files.download('Outputs_visualization.zip')